### Topic Modeling - Twitter data

#### Imports and Init

In [ ]:
%%capture
!pip install persim
!pip install pyshp
!pip install kde-gpu
!pip install pyLDAvis==2.1.2
!pip install pandas
!pip install nltk
!pip install gensim

!pip install spacy
# !pip install pyLDAvis --upgrade

!python -m spacy download en_core_web_sm

%matplotlib inline

In [ ]:
import sys

In [ ]:
from pathlib import Path
import os

In [ ]:
from importlib import reload

In [ ]:
import pandas as pd
import pickle
import gensim.corpora as corpora# Create Dictionary
import pyLDAvis
import gensim

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


#### Read processed Twitter Data

In [ ]:
path_to_base = "/content/drive/Shareddrives/taxi_trends/Experiments/Twitter Data/Topic Modeling"
base_path = Path(path_to_base)
os.chdir(base_path)
print(os.listdir())

import topic_model

['LDA_snapshots', 'topic_model.py', 'twitter_processed_zonefiltered.csv', 'data_lemmatized_octfinal31102021175154.pkl', '.ipynb_checkpoints', '__pycache__', 'Twitter Daily Analysis.ipynb', 'Twitter Dataprep.ipynb']


/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


In [ ]:
data_path = Path('/content/drive/Shareddrives/taxi_trends/Data/twitter data/')

# Load saved data
with open(data_path / 'data_lemmatized_3mon.pkl', 'rb') as inp:
  data_lemmatized = pickle.load(inp)

twitter_df = pd.DataFrame(data_lemmatized)

#### Building the LDA model

In [ ]:
# Subsetting for a day
twitter_df_sub = twitter_df[twitter_df['tw_date'] == 20181225]
twitter_df_sub.shape

(238, 7)

In [ ]:
print(list(twitter_df_sub['tw_text_lem'].head()))

data_lemmatized = list(twitter_df_sub['tw_text_lem'])

[['east', 'street', 'come', 're', 'contact', 'email', 'twitt', 'assistance', 'sorry', 'capturing', 'holidayparty', 'wish', 'delicious', 'merry', 'choose', 'celebrate', 'unabashed'], ['love', 'love', 'solshine', 'thank', 'good', 'family', 'sound', 'earlyscreene', 'otherhalfnyc', 'bronze', 'street', 'king', 'queen', 'fineart', 'bronze', 'nightout', 'bar', 'latina', 'nightout', 'truefriend', 'bar', 'theeightfour', 'get', 'fantasy', 'legday', 'smell', 'bad', 'public', 'today', 'celebrate', 'see', 'special', 'thank', 'come', 'eat', 'drink', 'must', 'order', 'recognize', 'trap', 'lion', 'frigthen', 'wolve', 'mente', 'sin', 'construction', 'direction', 'clear', 'construction', 'direction', 'station', 'work', 'hard', 'never', 'stop', 'ilvin', 'ilvin', 'makemullahdj', 'djbashlive', 'shooting', 'shotingday', 'lasalanyc', 'nice', 'ambience', 'quaint', 'bar', 'great', 'setting', 'get', 'catering', 'art', 'light', 'healthy', 'bless', 'blend', 'astoria', 'go', 'movingimagenyc', 'yesterday', 'almost'

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]]


In [ ]:
num_topics = 5

In [ ]:
# 45 mins to run for raw tweets
import os, warnings
warnings.simplefilter("ignore")
print("Building LDA model")
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            # update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            # alpha='auto',
                                            per_word_topics=True)

from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# id2word, corpus, lda_model = topic_model.buildLDA(data_lemmatized, k=10)

Building LDA model
[(0,
  '0.021*"incident" + 0.021*"clear" + 0.020*"street" + 0.020*"bus" + '
  '0.011*"line" + 0.009*"station" + 0.005*"time" + 0.005*"today" + 0.005*"get" '
  '+ 0.004*"go"'),
 (1,
  '0.039*"station" + 0.034*"clear" + 0.028*"direction" + 0.026*"construction" '
  '+ 0.025*"bus" + 0.015*"side" + 0.014*"line" + 0.010*"street" + '
  '0.009*"incident" + 0.007*"bind"'),
 (2,
  '0.032*"claro" + 0.012*"year" + 0.009*"love" + 0.009*"station" + 0.008*"go" '
  '+ 0.008*"time" + 0.006*"new" + 0.005*"get" + 0.005*"today" + 0.005*"see"'),
 (3,
  '0.060*"exit" + 0.011*"parkway" + 0.010*"street" + 0.010*"clear" + '
  '0.010*"incident" + 0.009*"ramp" + 0.009*"sugardaddysnyc" + '
  '0.007*"construction" + 0.006*"music" + 0.005*"main"'),
 (4,
  '0.017*"station" + 0.012*"go" + 0.012*"clear" + 0.011*"job" + 0.010*"year" + '
  '0.008*"get" + 0.007*"see" + 0.007*"make" + 0.006*"new" + 0.006*"love"')]


In [ ]:
# Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.2945498610261655


In [ ]:
# lda_model = gensim.models.lda_models.LDA.load("LDA_snapshots/2018_10")

print("Topics distribution")
doc_topic_dists = topic_model.get_doc_topic_dists(lda_model)
doc_lengths = [len(text) for text in data_lemmatized]

lda_model.print_topics()

Topics distribution


[(0,
  '0.021*"incident" + 0.021*"clear" + 0.020*"street" + 0.020*"bus" + 0.011*"line" + 0.009*"station" + 0.005*"time" + 0.005*"today" + 0.005*"get" + 0.004*"go"'),
 (1,
  '0.039*"station" + 0.034*"clear" + 0.028*"direction" + 0.026*"construction" + 0.025*"bus" + 0.015*"side" + 0.014*"line" + 0.010*"street" + 0.009*"incident" + 0.007*"bind"'),
 (2,
  '0.032*"claro" + 0.012*"year" + 0.009*"love" + 0.009*"station" + 0.008*"go" + 0.008*"time" + 0.006*"new" + 0.005*"get" + 0.005*"today" + 0.005*"see"'),
 (3,
  '0.060*"exit" + 0.011*"parkway" + 0.010*"street" + 0.010*"clear" + 0.010*"incident" + 0.009*"ramp" + 0.009*"sugardaddysnyc" + 0.007*"construction" + 0.006*"music" + 0.005*"main"'),
 (4,
  '0.017*"station" + 0.012*"go" + 0.012*"clear" + 0.011*"job" + 0.010*"year" + 0.008*"get" + 0.007*"see" + 0.007*"make" + 0.006*"new" + 0.006*"love"')]

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.049478 -0.030231       1        1  35.933464
2     -0.117396  0.029313       2        1  27.231120
1      0.066962 -0.087951       3        1  20.478784
0      0.023297 -0.019244       4        1  11.860260
3      0.076615  0.108113       5        1   4.496372, topic_info=             Term        Freq       Total Category  logprob  loglift
90           exit   65.000000   65.000000  Default  30.0000  30.0000
509         claro  151.000000  151.000000  Default  29.0000  29.0000
354           bus  141.000000  141.000000  Default  28.0000  28.0000
29          clear  256.000000  256.000000  Default  27.0000  27.0000
30   construction  111.000000  111.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
841         party    1.897622   15.529069   Topic5  -5.9975   0.9998
81          thank    2.644042   65.770290   Topic5  -5.6658  -0.1120
42             go    2.624079  143.145837   Topic5  -5.6733  -0.8972
239          time    2.138464   99.526211   Topic5  -5.8780  -0.7384
4            come    1.972840   74.870291   Topic5  -5.9586  -0.5344

[354 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
2839      2  0.815491          airhorn
4033      4  0.485937  allouchegallery
468       1  0.352091           always
468       2  0.406259           always
468       3  0.054168           always
...     ...       ...              ...
112       1  0.442807             year
112       2  0.391993             year
112       3  0.130664             year
112       4  0.021777             year
112       5  0.007259             year

[579 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 1, 4])

In [ ]:
# Hyperparameter tuning
from gensim.models import CoherenceModel

# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
# Ref: https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
# Hyperparameter tuning
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
  'Topics': [],
  'Alpha': [],
  'Beta': [],
  'Coherence': []
}
                
                
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)

    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  0%|          | 0/540 [00:43<?, ?it/s]

100%|██████████| 540/540 [22:20<00:00,  2.48s/it]


In [ ]:
lda_model.save('LDA_snapshots/20181225_lda')

In [ ]:
lda_model = gensim.models.LdaModel.load('LDA_snapshots/full_oct')

In [ ]:
lda_model.show_topics()

## Dynamic Topic models using LDASeq

In [ ]:
# setting up our imports

from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger

In [ ]:
time_slices = [val for val in twitter_df.tw_date.value_counts().sort_index()]

In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=id2word, time_slice=time_slices, num_topics=5, passes=10, )

In [ ]:
ldaseq.save('LDA_snapshots/3_mon_lda_seq')